In [ ]:
import os
from pprint import pprint
from ast import literal_eval

import aiohttp
from dotenv import load_dotenv
load_dotenv()

In [ ]:
GITHUB_API_KEY = os.getenv('API_KEY')
HEADERS = {
    'Authorization': f'Bearer {GITHUB_API_KEY}',
    'Accept': 'application/vnd.github+json',
    "X-GitHub-Api-Version": "2022-11-28"
}

MY_NAME = os.getenv('MY_NAME')
WHITELIST = set(literal_eval(os.getenv('WHITELIST')))

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://api.github.com/user/repos', headers=HEADERS, params={"name": MY_NAME}) as resp:
        print(resp.status)
        pprint(await resp.json())

In [ ]:
params = {
    "per_page": 100
}

async with aiohttp.ClientSession() as session:
    async with session.get('https://api.github.com/user/followers', headers=HEADERS, params=params) as resp:
        followers = await resp.json()
    
    async with session.get('https://api.github.com/user/following', headers=HEADERS, params=params) as resp:
        following = await resp.json()

In [ ]:
len(followers), len(following) 

In [ ]:
followers_list = [follower['login'] for follower in followers]
following_list = [follow['login'] for follow in following]

In [ ]:
set(followers_list) - set(following_list)

In [ ]:
list(set(following_list) - set(followers_list) - WHITELIST)

In [ ]:
async with aiohttp.ClientSession() as session:
    for user in list(set(following_list) - set(followers_list) - WHITELIST):
        async with session.delete(f'https://api.github.com/user/following/{user}', headers=HEADERS) as resp:
            print(resp.status)
            print(f"unfollow username: {user}")